In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy

from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import dijkstra

In [3]:
STEP2_RESULTS = './data/Rat_info_072023/MRGvsM_results_08272023/MRGvsMxeL1000_corrES_wtargets_sortedbyPearson-duplicatesdropped_plt01_08272023.csv'

In [6]:
z = pd.read_csv(STEP2_RESULTS, index_col=0)

In [8]:
z.head(5)

,sig_id,pert_type,pearson,spearman,ES,pert_name,target
0,CPC001_VCAP_24H_N13_SB-205607_10uM,ChemicalPert,0.631691,0.525836,0.347249,SB-205607,OPRD1
1,LKCP002_U2OS_48H_H04_cilengitide_10uM,ChemicalPert,0.599193,0.521385,0.285579,cilengitide,"ITGAV, ITGB3"
2,LJP007_MCF10A_24H_A11_XMD-885_0.12uM,ChemicalPert,0.597751,0.476661,0.363378,XMD-885,"MAPK7, LRRK2"
3,ASG003_U2OS_6H_M24_YM-155_0.12uM,ChemicalPert,0.597642,0.533652,0.288425,YM-155,BIRC5
4,CPC006_PL21_6H_A06_temozolomide_10uM,ChemicalPert,0.582085,0.543747,0.328273,temozolomide,MGMT


In [9]:
COMP_FILE = './rat_col_MRGvMupdowngenes_07182023.csv'
target_gene = 'OPRD1' # Selected from top pearson correlation score of previous file and reasoning based on biological evidence 
NETWORK_FILE = '../brain_USP7/brain.txt'
NAME = 'MRGvsM'
DATE = '08272023'

In [10]:
# Load in comp set
comp_set = pd.read_csv(COMP_FILE) # Insert sel_col file from step 1
comp_set = comp_set.rename(columns={'Unnamed: 0': 'Gene'})
comp_genes = set(comp_set['Gene'].unique())

# Load in brain network data
network = pd.read_csv(NETWORK_FILE, sep='\t', header=None)
network.columns = ['S', 'E', 'weight']

# Convert weight to distance
#10.0x(1.0-weight)
network['distance'] = 10*(1-network['weight'])


# Create graph matrix
# Get unique nodes
nodes = np.unique(network[['S', 'E']].values)

# Create a dictionary to map node names to indices
node_to_index = {node: i for i, node in enumerate(nodes)}

# Initialize an empty adjacency matrix
adj_matrix = np.zeros((len(nodes), len(nodes)))

In [11]:
# Populate the adjacency matrix with edge weights
mag = 1000000
for i in range(0,int(len(network) / mag)+1):
    for _, row in network.iloc[i*mag:(i+1)*mag,].iterrows():
        start_index = node_to_index[row['S']]
        end_index = node_to_index[row['E']]
        adj_matrix[start_index, end_index] = row['distance']
        
# Convert the adjacency matrix to a sparse matrix
sparse_graph = csr_matrix(adj_matrix)
print(sparse_graph.shape) 

(17180, 17180)


In [12]:
# Run Djikstras starting at target gene
print(target_gene, 'node:', node_to_index[target_gene])

dist_matrix, predecessors = dijkstra(csgraph=sparse_graph, directed=False, indices=node_to_index[target_gene], return_predecessors=True)

# Convert graph to dataframe and format
ddf = pd.DataFrame(dist_matrix)
ddf.index = list(node_to_index.keys())

OPRD1 node: 9871


# Check f_gene and graph node overlap then get scores

In [13]:
comp_genes_g = set(comp_genes).intersection(set(ddf.index))
print(len(comp_genes_g))
print(comp_genes_g)

20
{'AKAP8L', 'HOMER2', 'HTRA1', 'ANKRD10', 'OXCT1', 'PNP', 'AURKB', 'BAG3', 'PGM1', 'ATF5', 'COL1A1', 'CIRBP', 'SLC1A4', 'ALDOC', 'TERF2IP', 'AKAP8', 'SLC27A3', 'PHKB', 'MYO10', 'S100A13'}


# Find the subnetwork of nodes from comp set

In [14]:
comp_idxs = [node_to_index[x] for x in comp_genes_g]
target_nodes = comp_idxs
source_node = node_to_index[target_gene]

subnetwork_nodes = set([source_node] + target_nodes)  # Include source and target nodes
for node in target_nodes:
    # Backtrack from the target node to the source node and include all intermediate nodes
    current_node = node
    while current_node != source_node:
        subnetwork_nodes.add(current_node)
        current_node = predecessors[current_node]

In [15]:
# Now you have a set of nodes that make up the subnetwork of shortest paths from the source node to the target nodes.
# You can use this set to extract the subgraph from your original graph representation.
subgraph_nodes = [nodes[node_index] for node_index in subnetwork_nodes]
print('Number of subnetwork nodes:', len(subnetwork_nodes),'\n')
print(subgraph_nodes)

Number of subnetwork nodes: 52 

['PGM1', 'CD44', 'OPRD1', 'SERPINE1', 'OPRM1', 'CDKN1A', 'MAPK14', 'CCNA2', 'BAG3', 'CCND1', 'RB1', 'SLC27A3', 'PHKB', 'AK4', 'PCNA', 'PARP1', 'OXCT1', 'AKAP12', 'SRC', 'GEM', 'EGFR', 'S100A13', 'MYO10', 'ATF2', 'HOMER2', 'AKAP8L', 'ATF5', 'COL1A1', 'CIRBP', 'SLC1A4', 'TERF2IP', 'AKAP8', 'WSB1', 'S100A6', 'ERBB3', 'S100A11', 'COL4A2', 'ADRB2', 'AURKB', 'FN1', 'VEGFA', 'TPM1', 'SFPQ', 'SRSF2', 'FOS', 'ANKRD10', 'PLAU', 'HTRA1', 'ALDOC', 'PDGFA', 'PDGFRA', 'PNP']


# FIND FILTERING METHOD FOR SHORTEST PATHS INSTEAD OF JUST FILTERING SUBGRAPH ON SOURCE AND END

In [16]:
network_filt = network[network['S'].isin(subgraph_nodes)]

In [17]:
network_filt = network_filt[network_filt['E'].isin(subgraph_nodes)]

In [18]:
network_filt.sort_values('distance')

,S,E,weight,distance
309247,PCNA,CCNA2,0.9804,0.196
1878356,CCND1,CDKN1A,0.9780,0.220
1931805,ATF2,FOS,0.9744,0.256
3644509,MAPK14,ATF2,0.9724,0.276
2246071,PDGFRA,PDGFA,0.9679,0.321
...,...,...,...,...
1661250,AKAP12,BAG3,0.5083,4.917
1961841,SFPQ,PCNA,0.5058,4.942
1088284,ANKRD10,OPRD1,0.5035,4.965
3657790,SERPINE1,COL4A2,0.5017,4.983


In [19]:
network_filt.describe()

,weight,distance
count,293.000000,293.000000
mean,0.785804,2.141959
std,0.132155,1.321549
min,0.501600,0.196000
25%,0.661500,1.077000
50%,0.820000,1.800000
75%,0.892300,3.385000
max,0.980400,4.984000


In [20]:
print('Graph file saved at: ' + target_gene + '_' + NAME + '_genes_subgraph_edges_' + DATE + '.csv')
network_filt.to_csv(target_gene + '_' + NAME + '_genes_subgraph_edges_' + DATE + '.csv')

Graph file saved at: OPRD1_MRGvsM_genes_subgraph_edges_08272023.csv
